In [1]:
%cd ..

d:\Blockchain\data\orai\orchai_validator


In [2]:
from pyspark.sql import SparkSession, DataFrame, Window
import pyspark.sql.functions as F

In [3]:
spark = SparkSession.builder.appName("Python Spark SQL basic example").getOrCreate()

23/01/18 06:58:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
def preprocess(df):
    num_val_blocks_df = (
            df.groupBy("block_height")
            .agg({"operator_address": "count"})
            .withColumnRenamed("count(operator_address)", "validators_count_per_block")
        )

In [68]:
df = spark.createDataFrame(
    [
        ("a",1,200,1,0),
        ("a",2,5,0,0),
        ("a",3,2,1,0),
        ("a",4,2,0,0),
        ("a",5,2,0,0),
        ("a",6,2,1,0),
        ("a",7,2,0,1),
        ("a",8,2,1,0),
        ("a",9,2,0,0),
        ("a",10,2,1,0),
        ("b",1,4,0,0),
        ("b",2,4,1,0), 
        ("b",3,20,0,0), 
        ("b",4,12,1,0), 
        ("b",5,12,0,0), 
        ("b",6,4,1,0), 
        ("b",7,4,1,0), 
        ("b",8,10,0,0), 
        ("b",9,4,1,0), 
        ("b",10,4,0,0),
        ("c",1,4,0,1),
        ("c",2,4,1,0), 
        ("c",3,4,1,0), 
        ("c",4,4,1,0),
        ("c",5,7,0,0),
        ("c",6,4,1,0), 
        ("c",7,4,0,0), 
        ("c",8,4,1,0), 
        ("c",9,4,1,0), 
        ("c",10,4,0,0),
        ("d",1,4,1,0),
        ("d",2,4,1,0), 
        ("d",3,4,1,0), 
        ("d",4,4,1,0),
        ("d",5,7,0,1),
        ("d",6,4,1,0), 
        ("d",7,4,0,0), 
        ("d",8,4,1,0), 
        ("d",9,4,1,0), 
        ("d",10,4,1,0),
        ("e",1,4,1,0),
        ("e",2,4,1,0), 
        ("e",3,4,1,0), 
        ("e",4,4,1,0),
        ("e",5,7,1,0),
        ("e",6,4,1,1), 
        ("e",7,4,1,0), 
        ("e",8,4,0,0), 
        ("e",9,4,1,0), 
        ("e",10,4,1,0),
    
    ],

    ["operator_address", "block_height", "tokens", "vote", "propose"]
    
    
    
    
    
    
    )

In [52]:
df.show(200)

+----------------+------------+------+----+-------+
|operator_address|block_height|tokens|vote|propose|
+----------------+------------+------+----+-------+
|               a|           1|   200|   1|      0|
|               a|           2|     5|   1|      0|
|               a|           3|     2|   1|      0|
|               a|           4|     2|   1|      0|
|               a|           5|     2|   1|      0|
|               a|           6|     2|   1|      0|
|               a|           7|     2|   1|      1|
|               a|           8|     2|   1|      0|
|               a|           9|     2|   1|      0|
|               a|          10|     2|   1|      0|
|               b|           1|     4|   1|      0|
|               b|           2|     4|   1|      0|
|               b|           3|    20|   1|      0|
|               b|           4|    12|   1|      0|
|               b|           6|     4|   1|      0|
|               b|           7|     4|   1|      0|
|           

In [53]:
max_token_df = (
            df.groupBy("block_height")
            .agg({"tokens": "max"})
            .withColumnRenamed("max(tokens)", 
            "max_tokens_per_block"))
df = df.join(max_token_df, on = "block_height", how = "left").orderBy("block_height")


In [14]:
df.show(1000)

+------------+----------------+------+-------------------------+--------------------+
|block_height|operator_address|tokens|total_token_amt_per_block|max_tokens_per_block|
+------------+----------------+------+-------------------------+--------------------+
|           1|               d|     4|                        1|                 200|
|           1|               e|     4|                        1|                 200|
|           1|               f|     4|                        1|                 200|
|           1|               g|     4|                        1|                 200|
|           1|               a|   200|                        1|                 200|
|           1|               b|     4|                        1|                 200|
|           1|               c|     4|                        1|                 200|
|           2|               e|     4|                        1|                   5|
|           2|               a|     5|                

In [69]:
def vote_proposed_score(df: DataFrame, vote_proposed_win_size: int):
        # df = df.withColumn("vote_propose_point", vote_score * df.vote + propose_score * df.propose)
        # #vote, propose: TRue, False

        size = vote_proposed_win_size-1

        # vote_propose_score_df = df.select("operator_address", "block_height", "vote_propose_point").orderBy(
        #     "operator_address", "block_height"
        # )
        

        cummulative_window = Window.partitionBy("operator_address").orderBy("block_height").rangeBetween(-size, 0)
        ### Window for moving average step
        ### Step now row and last "size" row

        lag_window = Window.partitionBy("operator_address").orderBy("block_height")
        ### Window for lag to check the null

        df = df.withColumn(
            ### Using lag to get the null - rows that do not change
            "vote_score",
            F.when(
                F.lag("vote", size).over(lag_window).isNotNull(), F.sum("vote").over(cummulative_window)
            ),
        )

        # df = df.join(vote_propose_score_df, on=["block_height", "operator_address"], how="left").orderBy("block_height")
        # df = df.drop("vote_propose_point")
        df = df.na.drop()

        max_vote_score_df = (
            df.groupBy("block_height")
            .agg({"vote_score": "max"})
            .withColumnRenamed("max(vote_score)", "max_vote_score")
        )

        df = df.join(max_vote_score_df, on="block_height", how="left").orderBy("block_height")
        df = df.withColumn("vote_score", df.vote_score / df.max_vote_score)
        df = df.drop("max_vote_score")
        df = df.drop("vote")
        df = df.drop("propose")

        return df


In [70]:
df = vote_proposed_score(df, 3).orderBy("operator_address", "block_height")

In [71]:
df.show(200)

+------------+----------------+------+----+-------+------------------+
|block_height|operator_address|tokens|vote|propose|        vote_score|
+------------+----------------+------+----+-------+------------------+
|           3|               a|     2|   1|      0|0.6666666666666666|
|           4|               a|     2|   0|      0|0.3333333333333333|
|           5|               a|     2|   0|      0|0.3333333333333333|
|           6|               a|     2|   1|      0|0.3333333333333333|
|           7|               a|     2|   0|      1|0.3333333333333333|
|           8|               a|     2|   1|      0|               1.0|
|           9|               a|     2|   0|      0|               0.5|
|          10|               a|     2|   1|      0|0.6666666666666666|
|           3|               b|    20|   0|      0|0.3333333333333333|
|           4|               b|    12|   1|      0|0.6666666666666666|
|           5|               b|    12|   0|      0|0.3333333333333333|
|     

In [16]:
list = ["oraivaloper18tf4uwrkcd4qk87jz3n0ruhsdzeg3fmdsdaw04",
"oraivaloper102rpf06zhyrfps4qzvehqjtr7rh52us5z2959t",
"oraivaloper10z9f6539v0ge78xlm4yh7tddrvw445s6d7s2xq",
"oraivaloper1zwxeq68hunufljymhy0f0x7r2vvh9mu2hh8nt8",
"oraivaloper1yujklf8xlcngsdylgqj352pj9x3yvrrwf3spc8",
"oraivaloper1mrv57zj3dpfyc9yd5xptnz2tqfez9fss4c9r85",
"oraivaloper17zr98cwzfqdwh69r8v5nrktsalmgs5sa83gpd9",
"oraivaloper1n4xkr4k4rpynp020e8xfqwm2e95g7rzx7w6kgl",
"oraivaloper1nedr7579x9dp6s3rrt7n5a5s5qczpnnstanuk5",
"oraivaloper16spaytw7nhdrv9lar0quwj6eex8cw9pvmtynxd",
"oraivaloper1asz5wl5c2xt8y5kyp9r04v54zh77pq90qar7e8",
"oraivaloper1djm07np8dzyg4et3d7dqtr3692l80nggvl0edh",
"oraivaloper1ucx0gm8kca2zvyr9d39z249j62y2t8r0rwtmr6",
"oraivaloper1ch3ewye24zm094ygmxu5e4z7d0xre3vhthctpn",
"oraivaloper1w96q7rrd6edc3fpmw6zayzdf5ajzucrnw3pt3s",
"oraivaloper1v26tdegnk79edw7xkk2xh8qn89vy6qej6yhsev",
"oraivaloper1kh9vejqxqqccavtv2nf683mx0z85mfpd7q566q",
"oraivaloper1d2ths86a84rxpp7s9j64qgrypntw2pn7lndfgs",
"oraivaloper1xsptthm2ylfw0salut97ldfan2jt032nye7s00",
"oraivaloper1qv5jn7tueeqw7xqdn5rem7s09n7zletreera88",
"oraivaloper16e6cpk6ycddk6208fpaya7tmmardhvr7h40yqy",
"oraivaloper10n3fs6fkl4fp9dcsdfl2vl3ay7pk7snntfg7da",
"oraivaloper130jsl66rgss6eq7qur02yfr6tzppdvxglz7n7g",
"oraivaloper1yc9nysml8dxy447hp3aytr0nssr9pd9a47l7gx",
"oraivaloper109vcny07r3waj9sld4ejasjyal0rudskeax7uc",
"oraivaloper1h89umsrsstyeuet8kllwvf2tp630n77aymck78",
"oraivaloper1mxqeldsxg60t2y6gngpdm5jf3k96dnju5el96f",
"oraivaloper1rqq57xt5r5pnuguffcrltnvkul7n0jdxxdgey0",
"oraivaloper1uhcwtfntsvk8gpwfxltesyl4e28aalmq9v9z0x",
"oraivaloper1xesqr8vjvy34jhu027zd70ypl0nnev5euy9nyl",
"oraivaloper1cp0jml5fxkdvmajcwvkue9d0sym6s0vqly88hg",
"oraivaloper1u2344d8jwtsx5as7u5jw7vel28puh34q7d3y64",
"oraivaloper1f6q9wjn8qp3ll8y8ztd8290vtec2yxyxxygyy2",
"oraivaloper1h9gg3xavqdau6uy3r36vn4juvzsg0lqvszgtvc",
"oraivaloper14nz2pqskfv9kcez8u0a9gnnsgwjerzqxpmne0y",
"oraivaloper1m2d5uhr65p9vvlw2w29kajud5q529a76v22wyu",
"oraivaloper13ckyvg0ah9vuujtd49yner2ky92lej6nwjvrjv",
"oraivaloper12ru3276mkzuuay6vhmg3t6z9hpvrsnplm2994n",
"oraivaloper14vcw5qk0tdvknpa38wz46js5g7vrvut8ku5kaa"]

In [17]:
len(list)

39

In [15]:
list = [1,2]